# Script SAE15 - Traiter des données




---


Nom : N'Landou

Prénom : Lucken Sender

Groupe de TP : B1



---



In [1]:
####################################################################
# DEBUT DE CODE : SAE15 - TRAITER DES DONNEES                      #
#------------------------------------------------------------------#
# PARTIE 1 : IMPORTATIONS, INSTALLATIONS ET CONFIGURATIONS         #
# PARTIE 2 : RECUPERATION DES DONNEES VELIB EN OPENDATA            #
# PARTIE 3 : MISE SOUS FORME DE DATAFRAME                          #
# PARTIE 4 : ANALYSES STATISTIQUES                                 #
# PARTIE 5 : EXPORTATION DES MESURES STATISTIQUES POUR LE WEB      #
# PARTIE 6 : SPATIALISATION DES DONNEES, EXPORTATION POUR LE WEB   #
####################################################################

In [2]:
####################################################################
# PARTIE 1 : IMPORTATIONS, INSTALLATIONS ET CONFIGURATION          #
####################################################################
# modules natifs à importer
import sys                     # pour l'accès au commandes système
import json                    # pour manipuler le format JSON
from google.colab import drive # pour l'accès au drive
import pandas as pd            # pour extraire, calculer, publier
#-------------------------------------------------------------------
# modules externes à installer et à importer
!pip install geopandas
!pip install contextily
import datetime as dt
import matplotlib.pyplot as plt

import contextily as ctx        # pour l'utilisation de cartes géographiques
import geopandas as gpd         # pour la spatialisation des données
#-------------------------------------------------------------------
# montage du drive
drive.mount('/content/drive', force_remount=True)
#-------------------------------------------------------------------
# modules internes à importer
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/SAE15/tools')
import sae15_tools as tools 
import sae15_spec as spec
####################################################################

     |████████████████████████████████| 1.0 MB 17.6 MB/s 
     |████████████████████████████████| 15.4 MB 34.3 MB/s 
     |████████████████████████████████| 6.3 MB 39.8 MB/s 
     |████████████████████████████████| 19.3 MB 54.0 MB/s 
Mounted at /content/drive


In [3]:
####################################################################
# PARTIE 2 : RECUPERATION DES DONNEES VELIB EN OPENDATA            #
####################################################################
# récupération des données statiques (données d'information)
static = tools.loadVelibInformation()['data']['stations']
#-------------------------------------------------------------------
# récupération des données dynamiques (données de status)
dyn = tools.loadVelibStatus()['data']['stations']
####################################################################

In [4]:
####################################################################
# PARTIE 3 : MISE SOUS FORME DE DATAFRAME                          #
####################################################################
# dataframe de l'information pour les stations
static = pd.DataFrame(static)
#-------------------------------------------------------------------
# dataframe du status pour les stations
dyn = pd.DataFrame(dyn)
#-------------------------------------------------------------------
# fusion des deux DataFrames en un seul (merge)
merged = pd.merge(dyn, static)
####################################################################

In [5]:
####################################################################
# PARTIE 4 : ANALYSES STATISTIQUES                                 #
####################################################################

# Function that returns stats
def stats(df, key):
  try:
    data = df[key]
  except:
    print('La clé n\'existe pas')
    return 0
  count = data.count()
  min = data.min()
  max = data.max()
  mean = data.mean()
  std = data.std()
  dic = {'Sum':[count],'Minimum':[min],'Maximum':[max],'Mean':[mean],'Std':[std]}
  return pd.DataFrame(dic)

def getLatestDate(stations_df):
    timestamp = stations_df.last_reported.max()
    date = dt.datetime.fromtimestamp(timestamp).strftime('%d-%m-%Y %H:%M:%S')
    return date

# mesures statistiques sur la capacité des stations
print("Statistiques sur la Capacité des stations")
display(pd.DataFrame(spec.stats(merged, 'capacity')))
print("\n")
#-------------------------------------------------------------------
# mesures statistiques sur les vélos disponibles
print("Statistiques sur les vélos disponibles")
display(pd.DataFrame(spec.stats(merged, 'numBikesAvailable')))
print("\n")
#-------------------------------------------------------------------
# mesures statistiques sur les stands disponibles
print("Statistiques sur les stands disponibles")
display(pd.DataFrame(spec.stats(merged, 'numDocksAvailable')))
print("\n")
#-------------------------------------------------------------------
# mesures statistiques sur les vélos mécaniques (optionnel)

#-------------------------------------------------------------------
# mesures statistiques sur les vélos électriques (optionnel)

#-------------------------------------------------------------------
# assemblage de toutes les statistiques dans un seul DataFrame
cap = stats(merged, 'capacity').rename(columns={"Minimum":"Min For Capacity", "Maximum":"Max For Capacity","Mean":"Mean For Capacity","Std":"Std For Capacity"})
bikes = stats(merged, 'numBikesAvailable').rename(columns={"Minimum":"Min For Bikes", "Maximum":"Max For Bikes","Mean":"Mean For Bikes","Std":"Std For Bikes"})
docks = stats(merged, 'numDocksAvailable').rename(columns={"Minimum":"Min For Docks", "Maximum":"Max For Docks","Mean":"Mean For Docks","Std":"Std For Docks"})
mergedStats = pd.merge(pd.merge(cap, bikes), docks)
#-------------------------------------------------------------------
# affichage dans la console du DataFrame des statistiques assemblées
display(mergedStats)
####################################################################

Statistiques sur la Capacité des stations


,Sum,Minimum,Maximum,Mean,Std
0,1436,0,74,31.238162,12.008317




Statistiques sur les vélos disponibles


,Sum,Minimum,Maximum,Mean,Std
0,1436,0,63,11.100279,11.684099




Statistiques sur les stands disponibles


,Sum,Minimum,Maximum,Mean,Std
0,1436,0,67,18.610028,12.851649


,Sum,Min For Capacity,Max For Capacity,Mean For Capacity,Std For Capacity,Min For Bikes,Max For Bikes,Mean For Bikes,Std For Bikes,Min For Docks,Max For Docks,Mean For Docks,Std For Docks
0,1436,0,74,31.238162,12.008317,0,63,11.100279,11.684099,0,67,18.610028,12.851649


In [6]:
####################################################################
# PARTIE 5 : EXPORTATION DES MESURES STATISTIQUES POUR LE WEB      #
####################################################################
# mise en forme du DataFrame des statistiques avant exportation

#-------------------------------------------------------------------
# exportation du DataFrame des statistiques au format HTML
spec.exportStatistics(mergedStats, 'stats_f.html')
####################################################################

In [7]:
####################################################################
# PARTIE 6 : SPATIALISATION DES DONNEES, EXPORTATION POUR LE WEB   #
####################################################################
# transformation du DataFrame des stations Vélib en GeoDataFrame
geom = gpd.points_from_xy(merged["lon"], merged["lat"])
# Create a DataFrame with a geometry containing the Points
geo_data = gpd.GeoDataFrame(merged, crs="EPSG:4326", geometry=geom)

#-------------------------------------------------------------------
# détermination de la dernière date de mise à jour des données 
# latestDate = spec.getLatestDate(dyn) # NE MARCHE TOUJOURS PAS
latestDate = getLatestDate(dyn)
#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos avec date de mise à jour 
# et export web de la carte

#geo_data.plot(markersize=100 * geo_data.numBikesAvailable / (geo_data.numDocksAvailable + geo_data.numBikesAvailable), figsize=(15, 15))

# conversion de coordonnées : contextily utilise les coordonnées EPSG:3857
geo_data_with_map = geo_data.to_crs(epsg=3857)

# figure et axes graphiques
f, axes = plt.subplots(1, figsize=(15,15))

# affichage des stations gélocalisés
geo_data_with_map.plot(geo_data.num_bikes_available, markersize=100 * geo_data.numBikesAvailable / (geo_data.numDocksAvailable + geo_data.numBikesAvailable), cmap="YlOrRd", ax=axes)

# effacement des axes gradués
axes.set_axis_off()
axes.set_title('Mise à jour du '+latestDate)
# ajout du fond de carte correspondant aux coordonnées géographiques des stations
ctx.add_basemap(axes)

# EXPORT
plt.savefig('/content/drive/My Drive/Colab Notebooks/SAE15/web/data/bikesRateMap.svg')
# affichage forçé
plt.show()


#-------------------------------------------------------------------
# carte du taux de disponibilité des stands avec date de mise à jour
# et export web de la carte
# conversion de coordonnées : contextily utilise les coordonnées EPSG:3857
geo_data_with_map = geo_data.to_crs(epsg=3857)

# figure et axes graphiques
f, axes = plt.subplots(1, figsize=(15,15))

# affichage des stations gélocalisés
geo_data_with_map.plot(geo_data.num_docks_available, markersize=spec.availableDocksRate(dyn), cmap="YlOrRd", ax=axes)

# effacement des axes gradués
axes.set_axis_off()
axes.set_title('Mise à jour du '+latestDate)
# ajout du fond de carte correspondant aux coordonnées géographiques des stations
ctx.add_basemap(axes)

# EXPORT
plt.savefig('/content/drive/My Drive/Colab Notebooks/SAE15/web/data/docksRateMap.svg')
# affichage forçé
plt.show()

#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos mécaniques (optionnel) 
# et export web de la carte

#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos électriques (optionnel) 
# et export web de la carte

####################################################################

Output hidden; open in https://colab.research.google.com to view.

In [8]:
####################################################################
# FIN DE CODE : VERIFIER LA PUBLICATION SUR LE WEB                 #
####################################################################